In [1]:
import os
import sys
import shutil
import numpy as np
import h5py
import matplotlib.pyplot as plt
plt.rc('font', size=12.0)

import tensorflow as tf
from keras import layers
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

2024-11-12 22:04:34.744560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 22:04:34.756076: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 22:04:34.759673: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
tuner_dir = 'trials'

if os.path.exists(tuner_dir):
    shutil.rmtree(tuner_dir)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("Memory growth enabled for the first GPU.")
    except RuntimeError as e:
        print(f"Could not enable memory growth: {e}")
else:
    print("No GPUs found.")

Memory growth enabled for the first GPU.


I0000 00:00:1731467076.011842    6143 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731467076.104292    6143 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731467076.107383    6143 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [3]:
model = ['nl']
label = ['NL']
x_names = ['a', 'alpha', 'param_S', 'param_L', '', 'trans1',
           '', 'trans2', '', 'M_chirp_det', 'q', 'z_cdf']
for k in range(len(model)):
    if model[k] == 'mp' or model[k] == 'np':
        x_names[4], x_names[6], x_names[8] = 'exp1', 'exp2', 'exp3'
    else:
        x_names[4], x_names[6], x_names[8] = 'csq1', 'csq2', 'csq3'
y_names = ['I1', 'I2']

# Define file directory and load data
dir = '/home/anik/bamr/out/aff_inv/'
mchain = h5py.File(dir + 'nl_all', 'r')['markov_chain_0']

# Prepare X, Y, Z based on loaded data
x_ncols, y_ncols = len(x_names), len(y_names)
nrows, data = mchain['nlines'][0], mchain['data']
X, Y = np.zeros((x_ncols, nrows)), np.zeros((y_ncols, nrows))

for i in range(x_ncols):
    X[i] = data[x_names[i]]
for i in range(y_ncols):
    Y[i] = data[y_names[i]]

# Transpose X and Y to shape (nrows, ncols)
X, Y = X.T, Y.T

scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_Y = MinMaxScaler(feature_range=(0, 1))

# Fit-transform X
x = scaler_X.fit_transform(X)
y = scaler_Y.fit_transform(Y)

# Split the data for training and validation as required
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.2, random_state=42)
x_ts, x_vl, y_ts, y_vl = train_test_split(x_ts, y_ts, test_size=0.01, random_state=42)

In [4]:
class DNNHyperModel(kt.HyperModel):
    def build(self, hp):
        model = tf.keras.Sequential()
        
        model.add(layers.Input(shape=(x_ncols,)))
        num_layers = hp.Int('num_layers', 2, 3)

        for i in range(num_layers):
            units = hp.Int(f'units_{i}', min_value=16, max_value=256, step=16)
            model.add(layers.Dense(units, activation='relu'))

        model.add(layers.Dense(2, activation='sigmoid'))
        model.compile(optimizer='adam', loss='mse')
        
        return model

tuner = kt.RandomSearch(
    DNNHyperModel(),
    objective='val_loss',
    max_trials=500,
    executions_per_trial=1,
    directory='trials',
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1.0e-6, 
    patience=10
)

lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=5, 
    min_lr=1e-6
)

tuner.search_space_summary()

I0000 00:00:1731467076.155968    6143 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731467076.157932    6143 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731467076.159616    6143 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731467076.286644    6143 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 256, 'step': 16, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 256, 'step': 16, 'sampling': 'linear'}


In [5]:
tuner.search(
    x_tr, y_tr,
    epochs=200,
    validation_data=(x_ts, y_ts),
    batch_size=128,
    callbacks=[early_stop, lr_schedule]
)

Trial 492 Complete [00h 00m 10s]
val_loss: 8.851317397784442e-05

Best val_loss So Far: 4.164625352132134e-05
Total elapsed time: 01h 00m 31s

Search: Running Trial #493

Value             |Best Value So Far |Hyperparameter
2                 |3                 |num_layers
208               |224               |units_0
64                |224               |units_1
192               |256               |units_2

Epoch 1/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0151 - val_loss: 0.0027 - learning_rate: 0.0010
Epoch 2/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step - loss: 0.0022 - val_loss: 0.0015 - learning_rate: 0.0010
Epoch 3/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - loss: 0.0013 - val_loss: 8.4311e-04 - learning_rate: 0.0010
Epoch 4/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step - loss: 7.0654e-04 - val_loss: 5.1350e-04 - learning_rate: 0.0010
Epoch 5/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - loss: 4.8028e-04 - val_loss: 3.9442e-04 - learning_rate: 0.0010
Epoch 6/200
67

KeyboardInterrupt: 

In [7]:
tuner.results_summary(num_trials=20)

Results summary
Results in trials/untitled_project
Showing 20 best trials
Objective(name="val_loss", direction="min")

Trial 022 summary
Hyperparameters:
num_layers: 3
units_0: 224
units_1: 224
units_2: 256
Score: 4.164625352132134e-05

Trial 039 summary
Hyperparameters:
num_layers: 3
units_0: 112
units_1: 256
units_2: 176
Score: 4.472920409170911e-05

Trial 329 summary
Hyperparameters:
num_layers: 3
units_0: 208
units_1: 224
units_2: 128
Score: 4.9298811063636094e-05

Trial 357 summary
Hyperparameters:
num_layers: 3
units_0: 256
units_1: 240
units_2: 112
Score: 5.0553688197396696e-05

Trial 299 summary
Hyperparameters:
num_layers: 3
units_0: 256
units_1: 256
units_2: 128
Score: 5.106718163006008e-05

Trial 020 summary
Hyperparameters:
num_layers: 3
units_0: 224
units_1: 208
units_2: 256
Score: 5.279558172333054e-05

Trial 398 summary
Hyperparameters:
num_layers: 3
units_0: 240
units_1: 256
units_2: 144
Score: 5.2992945711594075e-05

Trial 132 summary
Hyperparameters:
num_layers: 3
uni

In [ ]:
# Retrieve the best model and evaluate
best_hps = tuner.get_best_hyperparameters(num_trials=5)[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.summary()

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1.0e-6, 
    patience=5
)

lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=5, 
    min_lr=1e-6
)
training = best_model.fit(x_tr, y_tr, epochs=1000, validation_data=(x_ts, y_ts), \
                          batch_size=128, callbacks=[early_stop, lr_schedule])

In [ ]:
plt.semilogy(training.history['loss'], ls='--', color='red', label='train')
plt.semilogy(training.history['val_loss'], color='orange', label='test')
plt.grid()
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.title("Training History")
plt.legend()
plt.show()

In [ ]:
# Assume y_pr contains the normalized predictions and y_vl is the validation data
y_pr = best_model.predict(x_vl)

# Initialize arrays to store denormalized values
Y_pr = scaler_Y.inverse_transform(y_pr)
Y_vl = scaler_Y.inverse_transform(y_vl)

# Plot the results
plt.plot(Y_vl[:, 0], label='I1 data')
plt.plot(Y_pr[:, 0], label='I1 pred')
plt.legend()
plt.show()

plt.plot(Y_vl[:, 1], label='I2 data')
plt.plot(Y_pr[:, 1], label='I2 pred')
plt.legend()
plt.show()
